# The AI Patent Analyst: From Unstructured PDFs to a Queryable Knowledge Graph

## 1. High-Level Summary

This project solves the critical challenge of analyzing unstructured patent PDFs by building an end-to-end pipeline that transforms them into a structured, queryable Knowledge Graph entirely within Google BigQuery.

The final solution is an interactive analysis engine that delivers significant cost savings by automating tasks that would otherwise require hundreds of hours of expensive expert analysis from patent lawyers or R&D engineers. It answers:

*   **Deep Architectural Analysis:** Use standard SQL with `UNNEST` and `GROUP BY` to discover the most common design patterns and technical component connections across hundreds of patents.

*   **Component Search:** Go beyond patent-level search to find specific, functionally similar technical parts across different domains (e.g., "find a mechanism for encrypting data").

*   **Quantitative Portfolio Analysis:** Compare patent applicants by the complexity (average component count) and breadth (number of domains) of their innovations.

## 2. The Workflow: A Multi-Stage AI Pipeline

Our solution follows a three-stage process, showcasing a powerful combination of BigQuery's multimodal, generative, and vector search capabilities.

### Stage 1: Multimodal Data Processing (🖼️ Pioneer)
We use **Object Tables** to directly read and process raw PDFs from Cloud Storage. The Gemini model is then used with `ML.GENERATE_TEXT` to analyze the both the text and the technical diagrams within the PDFs.

### Stage 2: Generative Knowledge Graph Extraction (🧠 Architect)
The consolidated patent text is fed into the `AI.GENERATE_TABLE` function. A custom prompt instructs the AI to act as an expert analyst, extracting a structured table of high-level insights (`invention_domain`, `problem_solved`) and a detailed, nested graph of all technical components, their functions, and their interconnections.

### Stage 3: Component-Level Semantic Search (🕵️‍♀️ Detective)
To enable deep discovery, we build a novel search engine that understands context. We use `ML.GENERATE_EMBEDDING` to create two separate vectors:
1.  One for the patent's high-level context (title, abstract)
2.  Another for each component's specific function

These vectors are mathematically averaged into a single, final vector for each component via BigQuery's UDF (User-Defined Functions).

Finally, `VECTOR_SEARCH` is used on these combined vectors, creating a powerful search that returns highly relevant, context-aware results.

## 3. Dataset Overview
- **403 PDFs** (197 English, others in FR/DE) at `gs://gcs-public-data--labeled-patents/*.pdf`.
- **Tables**: `extracted_data` (metadata), `invention_types` (labels), `figures` (91 diagram coordinates).
- **Source**: [Labeled Patents](https://console.cloud.google.com/marketplace/product/global-patents/labeled-patents?inv=1&invt=Ab5j9A&project=bq-ai-patent-analyst&supportedpurview=organizationId,folder,project) (1TB/mo free tier).

## 4. Code
*   **Notebook & Repository:** [https://github.com/veyselserifoglu/bq-ai-patent-analyst/blob/main/notebooks/bigquery-ai-the-patent-analyst-project.ipynb](https://github.com/veyselserifoglu/bq-ai-patent-analyst/blob/main/notebooks/bigquery-ai-the-patent-analyst-project.ipynb)

## 5. Architecture Pipeline

In [3]:
from IPython.display import HTML

# Display Architecture pipeline

HTML(f'''
<div style="text-align: center; padding: 15px;">
    <a href="https://github.com/veyselserifoglu/bq-ai-patent-analyst/blob/main/doc/Patent%20Analysis%20Pipeline%20Architecture%20-%20PNG.png?raw=true" 
       target="_blank" 
       style="cursor: pointer; display: inline-block; text-decoration: none;">
        <div style="position: relative; display: inline-block;">
            <img src="https://github.com/veyselserifoglu/bq-ai-patent-analyst/blob/main/doc/Patent%20Analysis%20Pipeline%20Architecture%20-%20PNG.png?raw=true" 
                 width="300" 
                 height="200"
                 style="border: 2px solid #e0e0e0; border-radius: 8px; transition: all 0.3s ease; box-shadow: 0 4px 8px rgba(0,0,0,0.1);"
                 onmouseover="this.style.borderColor='#4285F4'; this.style.boxShadow='0 6px 12px rgba(66, 133, 244, 0.3)'"
                 onmouseout="this.style.borderColor='#e0e0e0'; this.style.boxShadow='0 4px 8px rgba(0,0,0,0.1)'">
            <div style="position: absolute; top: 8px; right: 8px; background: rgba(255,255,255,0.9); border-radius: 50%; width: 24px; height: 24px; display: flex; align-items: center; justify-content: center; font-size: 14px;">
                ↗
            </div>
        </div>
    </a>
    <p style="margin-top: 12px; color: #5f6368; font-size: 13px; font-style: italic;">Click to explore the full architecture</p>
</div>
''')

In [15]:
# For visualization purposes
%pip install -q pyvis
%pip install -q plotly
%pip install -q ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
# BigQuery
import os
from google.cloud import bigquery
from kaggle_secrets import UserSecretsClient
import pandas as pd
from pyvis.network import Network
import plotly.express as px
from google.cloud import bigquery
from IPython.display import Image, display, HTML, IFrame
import ipywidgets as widgets
from ipywidgets import Layout

# Google Cloud Project Setup

This guide outlines the one-time setup required in Google Cloud and Kaggle to enable the analysis.

---

### 1. Google Cloud Project Configuration

First, configure your Google Cloud project.

1.  **Select or Create a Project**
    * Ensure you have a Google Cloud project.
    * Copy the **Project ID** (e.g., `my-project-12345`), not the project name.

2.  **Enable Required APIs**
    * In your project, enable the following two APIs:
        * **Vertex AI API**
        * **BigQuery Connection API**

3.  **Create a Service Account for the Notebook**
    * This service account allows the Kaggle notebook to act on your behalf.
    * Navigate to **IAM & Admin** > **Service Accounts**.
    * Click **+ CREATE SERVICE ACCOUNT**.
    * Give it a name (e.g., `kaggle-runner`).
    * Grant it these three roles: `Be sure to follow the principle of least privilege.`  
        * `BigQuery Admin`
        * `Vertex AI User`
        * `Service Usage Admin`
    * After creating the account, go to > manage keys > create a new key. A file will be downloaded to your computer.

---

### 2. Kaggle Notebook Configuration

Next, configure this Kaggle notebook to use your project.

1.  **Add Kaggle Secrets**
    * In the notebook editor, go to the **"Add-ons"** menu and select **"Secrets"**.
    * Add two secrets:
        * **`GCP_PROJECT_ID`**: Paste your Google Cloud **Project ID** here.
        * **`GCP_SA_KEY`**: Open the downloaded JSON key file, copy its entire text content, and paste it here.

---

### 3. Final Permission Step (After Running Code)

The first time you run the setup cells in the notebook, a new BigQuery connection will be created. This connection has its own unique service account that needs permission to use AI models.

1.  **Find the Connection Service Account**
    * After running the setup cells, go to **BigQuery** > **External connections** in your Google Cloud project.
    * Click on the connection named `llm-connection`.
    * Copy its **Service Account ID** (it will look like `bqcx-...@...gserviceaccount.com`).

2.  **Grant Permission**
    * Go to the **IAM & Admin** page.
    * Click **+ Grant Access**.
    * Paste the connection's service account ID into the **"New principals"** box.
    * Give it the single role of **`Vertex AI User`**.
    * Click **Save**.

---

With this setup complete, the notebook has secure access to your Google Cloud project and can run all subsequent analysis cells.

In [17]:
user_secrets = UserSecretsClient()
project_id = user_secrets.get_secret("GCP_PROJECT_ID")
gcp_key_json = user_secrets.get_secret("GCP_SA_KEY")
location = 'US'

In [18]:
# Write the key to a temporary file in the notebook's environment
key_file_path = 'gcp_key.json'
try:
    with open(key_file_path, 'w') as f:
        f.write(gcp_key_json)
    
    # Remove "> /dev/null 2>&1" to show the output.
    # Authenticate the gcloud tool using the key file
    !gcloud auth activate-service-account --key-file={key_file_path} > /dev/null 2>&1
    
    # Configure the gcloud tool to use your project
    !gcloud config set project {project_id} > /dev/null 2>&1
    
finally:
    # Securely delete the key file immediately after use
    if os.path.exists(key_file_path):
        os.remove(key_file_path)

# Enable the Vertex AI and BigQuery Connection APIs. Run only once Or Enable using the Cloud Interface.
# !gcloud services enable aiplatform.googleapis.com bigqueryconnection.googleapis.com > /dev/null 2>&1

In [ ]:
# This command creates the connection resource. Remove "> /dev/null 2>&1" to show the output.
!bq mk --connection --location={location} --connection_type=CLOUD_RESOURCE llm-connection > /dev/null 2>&1

In [ ]:
# This command shows the details of your connection. Remove "> /dev/null 2>&1" to show the output.
!bq show --connection --location={location} llm-connection > /dev/null 2>&1

# BigQuery Resource Creation

This section runs three commands to create the necessary resources for our analysis inside your BigQuery project.

---

### 1. Create a Dataset in the Correct Region.

First, we create a new dataset named `patent_analysis` in our chosen region. This dataset acts as a container for the AI model and the object table we will create next.

### 2. Create a Reference to the AI MultiModel.

Next, we create a "shortcut" to Google's `gemini-2.5-flash` model. This command gives us an easy name, `gemini_vision_analyzer`, to use in our analysis queries.

### 3. Create an Object Table for the PDFs.

Next, we create an object table named `patent_documents_object_table`. This is a special "map" that points directly to all the raw PDF files in the public Google Cloud Storage bucket, making them ready for analysis.

### 4. Create a Reference to the AI Embedding Model.

Next, we create a "shortcut" to Google's `gemini-embedding-001` model. This command gives us an easy name, `embedding_model`, to use in our embedding tasks.

### 5. Create a Reference to the AI Classifier Model.

Finally, we create a "shortcut" to Google's `gemini-2.5-flash` model. This command gives us an easy name, `gemini_text_classifier`, to use in our classification tasks.

---

In [19]:
# Initiate BigQuery client.
client = bigquery.Client(project=project_id, location=location)
client

In [28]:
# 1. Create the new dataset in the correct location
create_dataset_query = f"""
CREATE SCHEMA IF NOT EXISTS `{project_id}.patent_analysis`
OPTIONS(location = '{location}');
"""
print(f"Creating dataset 'patent_analysis' in {location}...")
job = client.query(create_dataset_query)
try:
    job.result()
    print("✅ Dataset created successfully or already exists.")
except Exception as e:
    print(f"❌ FAILED to create dataset. Error:\n\n{e}")


# 2. Create the AI model reference inside the new dataset
create_model_query = f"""
CREATE OR REPLACE MODEL `{project_id}.patent_analysis.gemini_vision_analyzer`
  REMOTE WITH CONNECTION `{location}.llm-connection`
  OPTIONS (endpoint = 'gemini-2.5-flash');
"""
print("\nCreating the AI model reference...")
job = client.query(create_model_query)
try:
    job.result() # This waits for the job to complete.
    print("✅ SUCCESS: Model 'gemini_vision_analyzer' created successfully.")
except Exception as e:
    print(f"❌ FAILED: The query failed. Please share this full error message:\n\n{e}")


# 3. Create the Object Table
# This query creates the "map" to the PDF files inside the local 'patent_analysis' dataset.
object_table_query = f"""
CREATE OR REPLACE EXTERNAL TABLE `{project_id}.patent_analysis.patent_documents_object_table`
WITH CONNECTION `{location}.llm-connection`
OPTIONS (
    object_metadata = 'SIMPLE',
    uris = ['gs://gcs-public-data--labeled-patents/*.pdf'] 
);
"""
print("Creating the object table...")
job = client.query(object_table_query)
try:
    job.result()
    print("✅ Object table created successfully.")
except Exception as e:
    print(f"❌ FAILED to create the object table. Error:\n\n{e}")


# 4. Create a remote connection for the embedding model.
sql_query = f"""
CREATE OR REPLACE MODEL `{project_id}.patent_analysis.embedding_model`
  REMOTE WITH CONNECTION `{location}.llm-connection`
  OPTIONS (endpoint = 'gemini-embedding-001');
"""
job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")


# 5. creates a helper function to perform L2 normalization on a vector.
create_classification_model = f"""
CREATE OR REPLACE FUNCTION `{project_id}.patent_analysis.L2_NORMALIZE`(vec ARRAY<FLOAT64>)
RETURNS ARRAY<FLOAT64> AS ((
  
  -- Calculate the L2 Norm (magnitude) of the vector.
  WITH vector_norm AS (
    SELECT SQRT(SUM(element * element)) AS norm
    FROM UNNEST(vec) AS element
  )
  
  -- Divide each element by the norm to create a unit vector.
  -- Handle the case where the norm is 0 to avoid division by zero errors.
  SELECT
    ARRAY_AGG(
      IF(norm = 0, 0, element / norm)
    )
  FROM
    UNNEST(vec) AS element, vector_norm
));
"""
print("Creating a Vector Normalization UDF...")
job = client.query(create_classification_model)
try:
    job.result()
    print("✅ Vector Normalization created successfully.")
except Exception as e:
    print(f"❌ FAILED to create the Vector Normalization. Error:\n\n{e}")


# 7. This creates a UDF (user defined function), a helper function for averaging vectors
sql_query = f"""
CREATE OR REPLACE FUNCTION `{project_id}.patent_analysis.VECTOR_AVG`(vec1 ARRAY<FLOAT64>, vec2 ARRAY<FLOAT64>)
    RETURNS ARRAY<FLOAT64>
    LANGUAGE js AS r'''
      if (!vec1 || !vec2 || vec1.length !== vec2.length) {{
        return null;
      }}
      let avg_vec = [];
      for (let i = 0; i < vec1.length; i++) {{
        avg_vec.push((vec1[i] + vec2[i]) / 2.0);
      }}
      return avg_vec;
    ''';
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

Creating a Vector Normalization UDF...
✅ Vector Normalization created successfully.


# Data Extraction & Knowledge Graph Creation

## What did we build?
We created two foundational data assets that power our analysis.

1. the `ai_text_extraction` table: transforms the raw PDFs into structured text, capturing the title and abstract.
2. the `patent_knowledge_graph` table: builds on this, creating a queryable graph of technical components and their connections.

## Why is this important?
- Automates Expert Work, saving hundreds of expert hours. 
- Accelerates Time-to-Insight, analyzing patents in seconds.

## How did we do it?
The process used a sequence of BigQuery's native AI functions:

1. **Multimodal Analysis**:
   - we used `ML.GENERATE_TEXT` to analyze the text and the technical diagrams within each patent's PDF.

2. **Knowledge Graph Extraction**:
   - Next, we fed all the consolidated text into the `AI.GENERATE_TABLE` function, to extract:
     - A nested table of all technical components.
     - Their functions.
     - Their connections for each patent.

In [ ]:
# 1. Multimodal Analysis - only texts - ai_text_extraction table

prompt_text = """From this patent document, perform the following tasks:

1.  **Extract these fields**: title, inventor, abstract, 
    the **Filed**, the **Date of Patent**, the international classification code, and the applicant.
    
2.  **Translate**: If the original title and abstract are in German or French, translate them into English.

3.  **Identify Language**: Determine the original language of the document.

Return ONLY a valid JSON object with EXACTLY these ten keys: 
"title_en", "inventor", "abstract_en", "filed", "date_of_patent", "class_international", "applicant", and "original_language".

**Formatting Rule**: For any key that has multiple values (like "inventor" or "class_international" or "applicant"), 
combine them into a single string, separated by a comma and a space. For example: "Igor Karp, Lev Stesin".

The "original_language" value must be one of these three strings: 'EN', 'FR', or 'DE'.
If any other field is unavailable, use null as the value.
"""

# The main SQL query.
sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.ai_text_extraction` AS (
  WITH raw_json AS (
      SELECT
        uri,
        ml_generate_text_llm_result AS llm_result
      FROM
        ML.GENERATE_TEXT(
          MODEL `{project_id}.patent_analysis.gemini_vision_analyzer`,
          TABLE `{project_id}.patent_analysis.patent_documents_object_table`,
          STRUCT(
            '''{prompt_text}''' AS prompt,
            2048 AS max_output_tokens,
            0.2 AS temperature,
            TRUE AS flatten_json_output
          )
        )
    ),
    parsed_json AS (
      -- Step 2: Clean and parse the JSON output.
      SELECT
        uri,
        llm_result,
        SAFE.PARSE_JSON(
          REGEXP_REPLACE(llm_result, r'(?s)```json\\n(.*?)\\n```', r'\\1')
        ) AS json_data
      FROM
        raw_json
    )
  SELECT
    uri,
    llm_result,
    
    SAFE.JSON_VALUE(json_data, '$.original_language') AS original_language,
    SAFE.JSON_VALUE(json_data, '$.title_en') AS extracted_title_en,
    SAFE.JSON_VALUE(json_data, '$.inventor') AS extracted_inventor,
    SAFE.JSON_VALUE(json_data, '$.abstract_en') AS extracted_abstract_en,
    SAFE.JSON_VALUE(json_data, '$.filed') AS filed_date,
    SAFE.JSON_VALUE(json_data, '$.date_of_patent') AS official_patent_date,
    SAFE.JSON_VALUE(json_data, '$.class_international') AS class_international,
    SAFE.JSON_VALUE(json_data, '$.applicant') AS applican
    
  FROM
    parsed_json
);
"""

job = client.query(sql_query)

try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [ ]:
# 1. Multimodal Analysis - only extending ai_text_extraction table with the technical diagrams.

diagram_prompt_text = """
Describe this technical diagram from a patent document. 
What is its primary function and what key components are labeled?
"""

sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.ai_text_extraction` AS (

  WITH figures_with_object_ref AS (
      SELECT
        fig.*, obj.ref
      FROM
        `bigquery-public-data.labeled_patents.figures` AS fig
      JOIN
        `{project_id}.patent_analysis.patent_documents_object_table` AS obj
      ON
        fig.gcs_path = obj.uri
    ),
    
    generated_descriptions AS (
      SELECT
        gcs_path,
        ml_generate_text_llm_result AS diagram_description
      FROM
        ML.GENERATE_TEXT(
          MODEL `{project_id}.patent_analysis.gemini_vision_analyzer`,
          (
            SELECT
              gcs_path,
              [
                JSON_OBJECT('uri', ref.uri, 'bounding_poly', [
                  STRUCT(x_relative_min AS x, y_relative_min AS y),
                  STRUCT(x_relative_max AS x, y_relative_min AS y),
                  STRUCT(x_relative_max AS x, y_relative_max AS y),
                  STRUCT(x_relative_min AS x, y_relative_max AS y)
                ])
              ] AS contents,
              '''{diagram_prompt_text}''' AS prompt
            FROM
              figures_with_object_ref
          ),
          STRUCT(
            4096 AS max_output_tokens,
            0.2 AS temperature,
            TRUE AS flatten_json_output
          )
        )
    ),

    aggregated_descriptions AS (
      SELECT
        gcs_path,
        ARRAY_AGG(diagram_description IGNORE NULLS) AS diagram_descriptions
      FROM
        generated_descriptions
      GROUP BY
        gcs_path
    )

  SELECT
    T.*,
    S.diagram_descriptions
  FROM
    `{project_id}.patent_analysis.ai_text_extraction` AS T
  LEFT JOIN
    aggregated_descriptions AS S
  ON
    T.uri = S.gcs_path
);
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [ ]:
# 2. Knowledge Graph - patent_knowledge_graph table.

# Define the schema as a Python variable
schema = """
invention_domain STRING, problem_solved STRING, patent_type STRING, 
components ARRAY<STRUCT<component_name STRING, component_function STRING, connected_to ARRAY<STRING>>>
"""

# The prompt text remains the same
prompt_text = """
From the following patent text, perform these tasks:
1. Determine the high-level technical domain (e.g., 'Telecommunications', 'Medical Devices').
2. Provide a one-sentence summary of the core problem the invention solves.
3. Classify the patent as a 'Method', 'System', 'Apparatus', or a combination.
4. Extract all technical components into a nested list. 
For each component, provide its name, its primary function, and a list of other components it is connected to.

Here is the text:
"""

sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.patent_knowledge_graph` AS (
  SELECT
    t.uri,
    t.invention_domain,
    t.problem_solved,
    t.patent_type,
    t.components
  FROM
    AI.GENERATE_TABLE(
      MODEL `{project_id}.patent_analysis.gemini_vision_analyzer`,
      (
        SELECT
          uri,
          CONCAT(
            '''{prompt_text}''',
            '\\n\\n',
            IFNULL(extracted_title_en, ''),
            '\\n\\n',
            IFNULL(extracted_abstract_en, ''),
            '\\n\\nDiagrams:\\n',
            IFNULL(ARRAY_TO_STRING(diagram_descriptions, '\\n'), '')
          ) AS prompt
        FROM
          `{project_id}.patent_analysis.ai_text_extraction`
        WHERE
          extracted_abstract_en IS NOT NULL
      ),
      STRUCT(
        '''{schema}''' AS output_schema
      )
    ) AS t
);
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

## Visualization

### Strategic Patent Portfolio Analysis

This section visualizes the data from our knowledge graph to uncover quantifiable insights about market trends and technical architecture.

----
### Chart 1: Patent Filing Trends by Technical Domain
It allows us to visually track market trends and identify which technology sectors experienced the most significant 

In [19]:
# This query creates a time-series of patent filings per domain.
sql_query = f"""
SELECT
  EXTRACT(YEAR FROM SAFE.PARSE_DATE('%d.%m.%Y', T1.filed_date)) AS filing_year,
  T2.invention_domain,
  COUNT(T1.uri) AS patent_count
FROM
  `{project_id}.patent_analysis.ai_text_extraction` AS T1
JOIN
  `{project_id}.patent_analysis.patent_knowledge_graph` AS T2
ON
  T1.uri = T2.uri
WHERE
  T1.filed_date IS NOT NULL
  AND T2.invention_domain IS NOT NULL
GROUP BY
  filing_year,
  invention_domain
ORDER BY
  filing_year,
  patent_count DESC;
"""

df_timeseries = client.query(sql_query).to_dataframe()

# Find the top 7 domains with the most patents overall.
top_domains = df_timeseries.groupby('invention_domain')['patent_count'].sum().sort_values(ascending=False).head(7).index.tolist()

# Group all other domains into a single "Other" category.
df_timeseries['display_domain'] = df_timeseries['invention_domain'].apply(
    lambda x: x if x in top_domains else 'Other'
)

# Aggregate the counts for the new display domains.
df_chart_data = df_timeseries.groupby(['filing_year', 'display_domain'])['patent_count'].sum().reset_index()


# Create the Interactive Stacked Area Chart.
fig = px.area(
    df_chart_data,
    x="filing_year",
    y="patent_count",
    color="display_domain",
    title="<b>Patent Filing Trends by Technical Domain Over Time</b>",
    labels={
        "filing_year": "Year of Filing",
        "patent_count": "Number of Patents Filed",
        "display_domain": "Invention Domain"
    },
    # Use a color scale that is easy to distinguish
    color_discrete_sequence=px.colors.qualitative.Vivid
)

# Customize the layout for a readable look
fig.update_layout(
    xaxis_title="<b>Year ➡️</b>",
    yaxis_title="<b>Annual Patent Count ⬆️</b>",
    legend_title="<b>Top Invention Domains</b>",
    font=dict(family="Arial, sans-serif", size=12)
)

fig.show()

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



### Chart 2: Architectural Blueprint of Inventions
It visualizes the total number of outgoing connections for the most central technical component hubs in our dataset.

In [45]:
import plotly.graph_objects as go
# This query prepares data for a Sankey diagram by finding the most common technical component connections.
sql_query = f"""
-- This query standardizes component names and finds the top 5 connections for each major hub.
WITH
  -- Step 1: Flatten all connections and standardize the names using a CASE statement.
  standardized_connections AS (
    SELECT
      CASE
        WHEN REGEXP_CONTAINS(LOWER(c.component_name), r'process|cpu') THEN 'Processor'
        WHEN REGEXP_CONTAINS(LOWER(c.component_name), r'mem|storage') THEN 'Memory'
        WHEN REGEXP_CONTAINS(LOWER(c.component_name), r'sens|camera') THEN 'Sensor'
        WHEN REGEXP_CONTAINS(LOWER(c.component_name), r'network|communicat') THEN 'Network Interface'
        WHEN REGEXP_CONTAINS(LOWER(c.component_name), r'bus') THEN 'Bus'
        WHEN REGEXP_CONTAINS(LOWER(c.component_name), r'user|display|input') THEN 'User Interface'
        WHEN REGEXP_CONTAINS(LOWER(c.component_name), r'server') THEN 'Server'
        ELSE NULL
      END AS source_hub,
      CASE
        WHEN REGEXP_CONTAINS(LOWER(connected_component), r'process|cpu') THEN 'Processor'
        WHEN REGEXP_CONTAINS(LOWER(connected_component), r'mem|storage') THEN 'Memory'
        WHEN REGEXP_CONTAINS(LOWER(connected_component), r'sens|camera') THEN 'Sensor'
        WHEN REGEXP_CONTAINS(LOWER(connected_component), r'network|communicat') THEN 'Network Interface'
        WHEN REGEXP_CONTAINS(LOWER(connected_component), r'bus') THEN 'Bus'
        WHEN REGEXP_CONTAINS(LOWER(connected_component), r'user|display|input') THEN 'User Interface'
        WHEN REGEXP_CONTAINS(LOWER(connected_component), r'server') THEN 'Server'
        ELSE NULL
      END AS target_component
    FROM
      `{project_id}.patent_analysis.patent_knowledge_graph` AS t,
      UNNEST(t.components) AS c,
      UNNEST(c.connected_to) AS connected_component
  ),

  -- Step 2: Count the frequency of each standardized connection.
  connection_counts AS (
    SELECT
      source_hub,
      target_component,
      COUNT(*) AS connection_count
    FROM
      standardized_connections
    WHERE
      source_hub IS NOT NULL AND target_component IS NOT NULL AND source_hub != target_component
    GROUP BY
      source_hub,
      target_component
  ),

  -- Step 3: Rank the connections for each source hub to find the top 5.
  ranked_connections AS (
    SELECT
      source_hub,
      target_component,
      connection_count,
      ROW_NUMBER() OVER(PARTITION BY source_hub ORDER BY connection_count DESC) as rank_num
    FROM
      connection_counts
  )

-- Final Step: Select the top 5 connections for each hub.
SELECT
  source_hub,
  target_component,
  connection_count
FROM
  ranked_connections
WHERE
  rank_num <= 5
ORDER BY
  source_hub,
  connection_count DESC;
"""

df_top_connections  = client.query(sql_query).to_dataframe()

fig = px.bar(
    df_top_connections,
    x="source_hub",
    y="connection_count",
    color="target_component", # Each color in the stack represents a connected component
    title="<b>Top 5 Connections for Major Component Hubs</b>",
    labels={
        "source_hub": "Component Hub",
        "connection_count": "Number of Connections",
        "target_component": "Connected To"
    },
    # Create a custom hover template for clarity
    hover_name="target_component",
    hover_data={
        "source_hub": False,
        "target_component": False,
        "connection_count": True
    }
)

# Customize the layout for a professional and readable look
fig.update_layout(
    xaxis_title="<b>Component Hub</b>",
    yaxis_title="<b>Total Outgoing Connections ⬆️</b>",
    xaxis={'categoryorder':'total descending'}, # Sort hubs by total connections
    legend_title="<b>Top 5 Connections</b>",
    font=dict(family="Arial, sans-serif", size=12)
)

fig.show()

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



# Patent Insights with SQL Analysis

## What did we build?

Now that we have transformed the unstructured patent data into a structured Knowledge Graph, we can finally ask it complex questions.

## Why is this important?
- This is the payoff. 
- We will run queries that are impossible to perform on the original text.
- Uncovering quantifiable insights about:
  - Invention complexity
  - Common design patterns across the entire dataset
- Proves the value of the data transformation pipeline.

## What will we find?
We will perform two types of analysis:

1. **Quantitative Analysis**:
   - Compare the average number of components across different technical domains
   - Measure and rank their complexity

2. **Architectural Pattern Mining**:
   - `UNNEST` the component data
   - Finds the most common "building blocks" and design patterns connected to any component we choose.

In [10]:
# Quantitative Analysis.

sql_query = f"""
SELECT
  invention_domain,
  COUNT(uri) AS number_of_patents,
  ROUND(AVG(ARRAY_LENGTH(components)), 2) AS average_components,
  MIN(ARRAY_LENGTH(components)) AS min_components,
  MAX(ARRAY_LENGTH(components)) AS max_components
FROM
  `{project_id}.patent_analysis.patent_knowledge_graph`
WHERE
  ARRAY_LENGTH(components) > 0
GROUP BY
  invention_domain
ORDER BY
  average_components DESC;
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

df = job.to_dataframe()
df[df['number_of_patents'] >= 4]

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,invention_domain,number_of_patents,average_components,min_components,max_components
48,Optoelectronics,4,8.00,5,10
65,Blockchain Technology,6,7.17,4,11
90,Electrical Engineering,5,6.20,4,9
121,Telecommunications,70,5.96,1,21
122,Image Processing,4,5.75,4,9
125,Information Technology,7,5.43,2,9
126,Wireless Communication,7,5.43,3,11
127,Medical Devices,24,5.42,2,10
130,Biotechnology,4,5.00,1,10
150,Wireless Communications,4,4.25,2,8


In [11]:
# Architectural Pattern Mining.

searching_topic = "server"

sql_query = f"""
WITH
  patent_components AS (
    SELECT
      t.uri,
      c.component_name,
      c.connected_to
    FROM
      `{project_id}.patent_analysis.patent_knowledge_graph` AS t,
      UNNEST(t.components) AS c
  ),

  component_connections AS (
    SELECT
      pc.uri,
      pc.component_name,
      connected_component
    FROM
      patent_components AS pc,
      UNNEST(pc.connected_to) AS connected_component
  )

SELECT
  connected_component,
  COUNT(connected_component) AS connection_count
FROM
  component_connections
WHERE
  REGEXP_CONTAINS(component_name, r'(?i){searching_topic}')
GROUP BY
  connected_component
ORDER BY
  connection_count DESC
LIMIT 10;
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

df = job.to_dataframe()
df.head(5)

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,connected_component,connection_count
0,User Device,3
1,Policy Evaluation Server,2
2,Boolean circuit,2
3,Client device,2
4,Network,2


# Patent Search Engine

## What did we build?
A powerful semantic search engine that finds specific technical components based on a natural language description of their function.

## Why is this important?
- Standard search finds keywords. This search finds meaning.
- By combining two different vector embeddings, the engine understands patent's components and the technical context in which it operates.
- This allows an engineer to find a "valve for precise fluid delivery" and get results from relevant medical patents, not car engine patents.

## How did we do it?
The process involves three key stages, all performed within BigQuery:

1. **Dual Embeddings**:
   - We first generate two separate vector embeddings:
     - One for the high-level patent context (title, abstract, domain, diagrams)
     - Another for the specific component's function

2. **Vector Combination**:
   - We then create a custom User-Defined Function (UDF) to mathematically average these two vectors.
   - This creates a single, final vector for each component that is rich with both specific and contextual meaning.

3. **Semantic Search**:
   - Finally, we use the `VECTOR_SEARCH` function to compare a user's query against these combined vectors.
   - Returns the most similar components from the entire dataset.


In [ ]:
# This query creates a flat table of all components from all patents.
sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.patent_components_flat` AS (
  SELECT
    t.uri,
    t.invention_domain,
    c.component_name,
    c.component_function,
    c.connected_to
  FROM
    `{project_id}.patent_analysis.patent_knowledge_graph` AS t,
    UNNEST(t.components) AS c
  WHERE
    c.component_function IS NOT NULL
    AND c.component_name IS NOT NULL
);
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [39]:
# This query creates a single context vector for each patent, reading from ai_text_extraction table.
sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.patent_context_embeddings` AS (
  SELECT
    t.uri,
    t.ml_generate_embedding_result AS patent_context_vector
  FROM
    ML.GENERATE_EMBEDDING(
      MODEL `{project_id}.patent_analysis.embedding_model`,
      (
        SELECT
          uri,
          CONCAT(
            'Represent this technical patent for semantic search: \\n\\n', 
            'Patent Title: ', IFNULL(extracted_title_en, ''), '\\n\\n',
            'Applicant: ', IFNULL(applican, ''), '\\n\\n',
            'International Classification: ', IFNULL(class_international, ''), '\\n\\n',
            'Abstract: ', IFNULL(extracted_abstract_en, ''), '\\n\\n',
            'Diagram Descriptions: ', IFNULL(ARRAY_TO_STRING(diagram_descriptions, '\\n'), '')
          ) AS content
        FROM
          `{project_id}.patent_analysis.ai_text_extraction`
        WHERE
          extracted_title_en IS NOT NULL
      )
    ) AS t
);
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [40]:
# This query creates a single specific function vector for each individual component.
sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.component_function_embeddings` AS (
  SELECT
    t.uri,
    t.component_name,
    t.ml_generate_embedding_result AS component_function_vector
  FROM
    ML.GENERATE_EMBEDDING(
      MODEL `{project_id}.patent_analysis.embedding_model`,
      (
        SELECT
          uri,
          component_name,
          CONCAT(
            'Represent this technical patent for semantic search: \\n\\n',
            'A component named "', component_name, '" whose function is to ', component_function
          ) AS content
        FROM
          `{project_id}.patent_analysis.patent_components_flat`
      )
    ) AS t
);
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [41]:
# Normalization

def normalize_and_save_vectors(
    table_id: str,
    vector_column: str,
    client: bigquery.Client
):
    """
   Normalizes a vector column in a BigQuery table in-place by replacing
    the table with its normalized version.

    Args:
        table_id: The full ID of the table to update (e.g., "project.dataset.table").
        vector_column: The name of the column containing the vectors to normalize.
        client: An authenticated BigQuery client object.
    """


    # This SQL query selects all original columns and replaces the vector
    # column with its normalized version.
    sql_query = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS (
      SELECT
        * EXCEPT({vector_column}),
        `{client.project}.patent_analysis.L2_NORMALIZE`({vector_column}) AS {vector_column}
      FROM
        `{table_id}`
    );
    """

    try:
        # Execute the query.
        job = client.query(sql_query)
        job.result()
    except Exception as e:
        print(f"❌ FAILED: An error occurred during normalization. Error:\n\n{e}")


# 1. Normalize the patent context embeddings.
normalize_and_save_vectors(
   table_id=f"{project_id}.patent_analysis.patent_context_embeddings",
   vector_column="patent_context_vector",
   client=client
)

# 2. Normalize the component function embeddings.
normalize_and_save_vectors(
   table_id=f"{project_id}.patent_analysis.component_function_embeddings",
   vector_column="component_function_vector",
   client=client
)

In [42]:
# This query joins all data and creates the final, context-aware combined vector.
sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.component_search_index` AS (
  SELECT
    flat.uri,
    flat.component_name,
    flat.component_function,
    `{project_id}.patent_analysis.VECTOR_AVG`(
      ctx.patent_context_vector,
      func.component_function_vector
    ) AS combined_vector
  FROM
    `{project_id}.patent_analysis.patent_components_flat` AS flat
  JOIN
    `{project_id}.patent_analysis.patent_context_embeddings` AS ctx
  ON
    flat.uri = ctx.uri
  JOIN
    `{project_id}.patent_analysis.component_function_embeddings` AS func
  ON
    flat.uri = func.uri AND flat.component_name = func.component_name
);
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

In [43]:
searching_prompt = "I want a similar patent to a small network IOT device"

sql_query = f"""
SELECT
  base.uri,
  base.component_name,
  base.component_function,
  distance
FROM
  VECTOR_SEARCH(
    TABLE `{project_id}.patent_analysis.component_search_index`,
    'combined_vector',
    (
      SELECT ml_generate_embedding_result
      FROM ML.GENERATE_EMBEDDING(
        MODEL `{project_id}.patent_analysis.embedding_model`,
        (SELECT '{searching_prompt}' AS content)
      )
    ),
    top_k => 7
  )
"""

job = client.query(sql_query)
try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")

df = job.to_dataframe()

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [47]:
def style_search_results(results_df: pd.DataFrame, search_query: str):
    """
    Takes a DataFrame of search results and returns a styled HTML table
    for clear and compelling presentation.
    """
    if results_df.empty:
        return "<p>⚠️ The search returned no results.</p>"

    # Create a copy to avoid modifying the original DataFrame
    styled_df = results_df.copy()

    # Ensure data is sorted by distance (lower is better)
    styled_df = styled_df.sort_values(by='distance', ascending=True)

    # Apply Formatting and Styling
    styled_df['uri'] = styled_df['uri'].apply(lambda x: f'<a href="{x}" target="_blank" style="color: #8ab4f8;">{x.split("/")[-1]}</a>')
    styled_df['distance'] = styled_df['distance'].map('{:.4f}'.format)

    # Improve table styling for readability
    styler = styled_df.style \
        .set_caption(f"<h3>Top 7 Component Matches for: '{search_query}'</h3>") \
        .set_properties(**{
            'text-align': 'left',
            'white-space': 'normal',
            'font-size': '14px',
            'font-family': 'Arial, sans-serif',
            'padding': '10px',
            'border': '1px solid #444',
        }) \
        .set_table_styles([
            {'selector': 'th', 'props': [
                ('text-align', 'left'), 
                ('font-size', '16px'), 
                ('font-weight', 'bold'), 
                ('background-color', '#333')]},
            {'selector': 'caption', 'props': [('caption-side', 'top'), 
                                              ('font-size', '18px'), 
                                              ('font-weight', 'bold'), 
                                              ('text-align', 'center')]},
            
            # Set specific column widths for better layout
            {'selector': 'td:nth-child(1)', 'props': [('width', '20%')]}, # uri
            {'selector': 'td:nth-child(2)', 'props': [('width', '20%')]}, # component_name
            {'selector': 'td:nth-child(3)', 'props': [('width', '45%')]}, # component_function
            {'selector': 'td:nth-child(4)', 'props': [('width', '15%')]}, # distance
        ]) \
        .hide(axis='index') \
        .background_gradient(subset=['distance'], cmap='RdYlGn_r')

    return styler.to_html()


def run_component_search(search_query: str, client: bigquery.Client):
    """
    Executes the BigQuery VECTOR_SEARCH and returns a styled HTML table.
    """
    # The VECTOR_SEARCH function already returns results sorted by distance.
    sql_query = f"""
    SELECT
      base.uri,
      base.component_name,
      base.component_function,
      distance
    FROM
      VECTOR_SEARCH(
        TABLE `{client.project}.patent_analysis.component_search_index`,
        'combined_vector',
        (
          SELECT ml_generate_embedding_result
          FROM ML.GENERATE_EMBEDDING(
            MODEL `{client.project}.patent_analysis.embedding_model`,
            (
              -- This is the corrected line with the task prefix
              SELECT CONCAT('Represent this technical patent component for semantic search: ', '{search_query}') AS content
            )
          )
        ),
        top_k => 7, 
        distance_type => 'COSINE'
      )
    """
    try:
        df = client.query(sql_query).to_dataframe()
        return style_search_results(df, search_query)
    except Exception as e:
        return f"<p>❌ FAILED: The query failed. Error: {e}</p>"

# Create the UI components
search_input = widgets.Text(
    value='a device for processing data',
    placeholder='Describe a technical function...',
    description='Search Query:',
    layout=Layout(width='80%')
)

search_button = widgets.Button(
    description='Find Similar Components',
    button_style='success',
    icon='search'
)

output_area = widgets.HTML(
    value="<p>Enter a query and click the button to see results.</p>"
)

def on_button_clicked(b):
    output_area.value = "<em>Searching...</em>"
    search_query = search_input.value
    html_result = run_component_search(search_query, client)
    output_area.value = html_result

search_button.on_click(on_button_clicked)

display(widgets.VBox([search_input, search_button, output_area]))
